# Problem 2 - Texture Synthesis
As discussed in class, the Efros and Leung algorithm synthesizes a new texture by performing an exhaustive search of a source texture for each synthesized pixel in the target image, in which sum-of-squared differences (SSD) is used to associate similar image patches in the source image with that of the target. The algorithm is initialized by randomly selecting a $3 \times 3$ patch from the source texture and placing it in the center of the target texture. The boundaries of this patch are then recursively filled until all pixels in the target image have been considered. Implement the Efros and Leung algorithm as the following Python function:
```
synthIm = synth_texture(sample, w, s)
```
where `sample` is the source texture image, `w` is the width of the search window, and `s=(ht, wt)` specifies the height and width of the target image `synthIm`. As described above, this algorithm will create a new target texture image, initialized with a $3 \times 3$ patch from the source image. It will then grow this patch to fill the entire image. As discussed in the textbook, when growing the image, unfilled pixels along the boundary of the block of synthesized values are considered at
each iteration of the algorithm. A useful technique for recovering the location of these pixels is morphological dilation, an operation that expands image regions. Specifically, we use `scipy.ndimage.binary_dilation`, `numpy.nonzero`, and/or `numpy.ix_` to recover the unfilled pixel locations along the boundary of the synthesized block in the target image. We have already provided some of the code for this part (which you should not need to modify).

In order to implement `synth_texture()`, you will first write a subroutine that for a given pixel in the target image, returns a list of matching patches in the source texture. This function should have the following syntax:
```
bestMatches = find_matches(template, sample, G)
```
where `bestMatches` is the list of matches, `template` is the $w \times w$ image template associated with a pixel of the target image, `sample` is the source texture image, and `G` is a 2D Gaussian mask discussed below. This routine is called by `synth_texture` and a pixel value is randomly selected from `bestMatches` to synthesize a pixel of the target image.

`bestMatches` is a list of patch locations represented by their center pixel coordinates. You should select patches based on both their absolute and relative SSD errors: to be included in `bestMatches`, the SSD error must be below a threshold $\delta$, and the SSD error must also be below the minimum SSD error times $(1+\epsilon)$. Efros and Leung use threshold values of $\epsilon = 0.1$ and $\delta = 0.3$.

Note that `template` can have values that have not yet been filled in by the image growing routine. Mask the template image such that these values are not
considered when computing SSD. Efros and Leung suggest using the following
image mask:
```
mask = G .* validMask
```
where `validMask` is a square mask of width $w$ that is 1 where the template is filled, 0 otherwise and `G` is a 2D zero-mean Gaussian with standard deviation $\sigma = w/6.4$ sampled on a $w \times w$ grid centered about its mean. `G` can be pre-computed using the `scipy.ndimage.gaussian_filter` routine or you can compute it yourself using the Gaussian equation. The purpose of the Gaussian is to down-weight pixels that are farther from the center of the template. You will need to normalize the mask such that its elements sum to 1.

In [ ]:
# @title Imports
%matplotlib inline
import numpy as np
from scipy import ndimage
import math
import cv2
import random
import sys
import math
import requests
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)

# Download images
r = requests.get("http://6.869.csail.mit.edu/sp22/pset7/rings.jpg", timeout=0.5)
if r.status_code == 200:
    with open("rings.jpg", 'wb') as f:
        f.write(r.content)

In [ ]:
# @title Helper functions
def im2double(im):
    info = np.iinfo(im.dtype) # Get the data type of the input image
    return im.astype(float) / info.max # Divide all values by the largest possible value in the datatype

def im2col_sliding(A, BSZ, stepsize=1):
    # This function is similar to the `im2col` function from MATLAB. It rearrange image blocks into columns.
    # Run the following line and check out the results to get more intuition
    # r = np.arange(25).reshape(5, 5); s = (3, 3); print(r); print(im2col_sliding(r, s).shape); print(im2col_sliding(r, s))
    m,n = A.shape
    s0, s1 = A.strides
    nrows = m-BSZ[0]+1
    ncols = n-BSZ[1]+1
    shp = BSZ[0],BSZ[1],nrows,ncols
    strd = s0,s1,s0,s1
    out_view = np.lib.stride_tricks.as_strided(A, shape=shp, strides=strd)
    return out_view.reshape(BSZ[0]*BSZ[1],-1)[:,::stepsize]

def sub2ind(array_shape, rows, cols):
    return rows*array_shape[1] + cols

# 4 points: implement find_matches()

In [ ]:
def find_matches(template: np.ndarray, sample: np.ndarray, G: np.ndarray):
    #TODO

    # The steps outlined below are just a suggestion. Feel free to deviate from
    # the recommended steps, but we may be less able to help you debug if you
    # do so.
    # parameters used by Efros and Leung, feel free to try your own
    epsilon = 0.1
    delta = 0.3

    ### Step 1: Create template mask
    # a) construct validMask as a square mask of width w that is 1 where template is filled
    # b) multiply validMask by G. Make sure the result is normalized to sum to 1.
    # c) unfold the result to a column vector (you can use x.ravel(order='F') for column-major order)

    ### Step 2: Vectorize `sample` and `template`
    # a) Partition `sample` into unfolded template-sized patches. Use im2col_sliding() to
    # create a 2D feature matrix of size [template.size, nBlocks] containing the
    # unfolded sample patches (you may need to perform separately for each RGB channel then concatenate)
    # b) Unfold `template` to a column vector (you can use x.ravel(order='F') for column-major order)
    # Side note:
    #   It would be more efficient to do this step once in synth_texture outside
    #   the for loop, but for pedagogy it is easier to think about find_matches()
    #   as a function of `sample` in its 3x3 form

    ### Step 3: Calculate SSD error
    # a) write vectorized code to calculate SSD error (remember to mask them) for all unfolded patches
    # b) find the best blocks according to the thresholds listed in the pdf
    # c) extract the center pixel value (3 numbers for RGB) of the best blocks
    # as best_matches. best_matches should have shape [3, number_of_candidates]

    raise NotImplementedError

# 2 points: implement synth_texture()


In [ ]:
def synth_texture(sample, w, s, verbose=False):
    # Texture Synthesis by Non-parameteric Sampling / Efros and Leung
    # This function computes the Gaussian kernel, then iteratively calls `find_matches`
    # and samples from the candidate RGB values to update the image.
    # You should be able to complete this function by only modifying parts marked NotImplemented
    # Turning verbose on will print out progress as the code runs

    # Normalize pixel intensity
    sample = im2double(sample)
    seed_size = 3
    [sheight, swidth, nChannels] = sample.shape
    theight, twidth = s
    synthIm = np.full((theight, twidth, nChannels), np.nan)

    # TODO: G is a centered 2D Gaussian with standard deviation w/6.4 sampled on a w x w grid
    # This is similar to what you did in an earlier pset.
    G = np.zeros((w, w))
    G = NotImplemented
    G = np.repeat(G[:, :, np.newaxis], nChannels, axis=2)

    # Initialization: pick a random 3x3 patch from sample and place in the middle of the synthesized image.
    # Just for convenience, keep some space (SEED_SIZE) from the boundary
    i0=31; j0=3
    c = [round(.5 * x) for x in s]
    synthIm[c[0]: c[0] + seed_size , c[1]: c[1] + seed_size ,:] = sample[i0: i0 + seed_size , j0: j0 + seed_size,:]

    # bitmap indicating filled pixels
    filled = np.zeros(s)
    filled[c[0]: c[0] + seed_size , c[1]: c[1] + seed_size ] = 1
    n_filled = int(np.sum(filled))
    n_pixels = s[0]*s[1]

    ### Main Loop
    next_p = n_pixels / 10
    while(n_filled < n_pixels):
        # report progress
        if(n_filled > next_p):
            if verbose:
                print(round(100 * n_filled / n_pixels), '% complete')
            next_p += n_pixels / 10

        ### dilate current boundary, find the next round of un-filled pixels
        ### (ii, jj) represents the locations
        border = ndimage.binary_dilation(filled).astype(filled.dtype) - filled
        ii, jj = np.where(border == 1)

        ### Permute (optional, to add random noise. feel free to play with it)
        #perm = np.random.permutation(len(ii))
        #ii = ii[perm]
        #jj = jj[perm]

        for i in range(len(ii)):
            ### Place window at the center of the current pixel to extract
            ### the template patch
            ic = [x for x in range(math.ceil(ii[i] - w/2), math.floor(ii[i] + w / 2)+1)]
            ic = np.asarray(ic)
            jc = [x for x in range(math.ceil(jj[i] - w/2), math.floor(jj[i] + w / 2)+1)]
            jc = np.asarray(jc)
            inbounds_ic = (ic >= 0) & (ic < theight)
            inbounds_jc = (jc >= 0) & (jc < twidth)
            template = np.full((w, w, nChannels), np.nan)

            nix_1 = np.ix_(np.nonzero(inbounds_ic)[0],np.nonzero(inbounds_jc)[0])
            nix_2 = np.ix_(ic[inbounds_ic],jc[inbounds_jc])
            template[nix_1] = synthIm[nix_2]

            ### Call find_matches() to get the best matches from the src image.
            best_matches = find_matches(template, sample, G)

            ### TODO: Sample randomly from best matches (uniformly) and update synthIm
            synthIm[ii[i], jj[i],:] = NotImplemented

            ### update bitmap indicating the corresponding pixel is filled
            filled[ii[i], jj[i]] = 1
            n_filled = n_filled + 1

    return synthIm

# 1 point: different window widths
Run your implementation using the grayscale source texture image `rings.jpg` available in the notebook, with window widths of $w = 5, 7, 13$, $s=[100, 100]$ and an initial starting seed of $(x, y) = (3, 31)$ (the initial pixel). It can be helpful to choose a smaller $s$ when debugging so that your code runs quickly. Make sure your notebook displays the synthesized textures for each of the three window sizes. Use pyplot subplots to organize your figures and label the widths.

In [ ]:
source = cv2.imread('rings.jpg')
w = 5
target = synth_texture(source, w, [100, 100])

# 1 point: interpret results
Explain the algorithm's performance with respect to window size. Write your response in a cell below.

# 2 points: reproducibility
For a given window size, if you re-run the algorithm with the same starting seed (i.e. the same initial pixel), do you get the same result? Why or why not?

Is this true for all window sizes?

Write your responses in a cell below (you do not necessarily need to plot results).